## Prepare Project
### Import Libraries and set up database connection

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Import pandas
import pandas as pd
pd.set_option('display.width', 2000)

# Import logging and surpress warnings
import logging
logging.getLogger("neo4j").setLevel(logging.ERROR)
logging.getLogger("pd").setLevel(logging.ERROR)

# Import Path
from pathlib import Path

# Import promg
from promg import Query

In [3]:
from util.db_helper_functions import get_db_connection, get_graph_statistics, clear_database, load_data

## Set up connection

In [4]:
conf_path = Path('bpic14', 'config.yaml')

In [5]:
db_connection = get_db_connection(conf_path)

These are the credentials that I expect to be set for the database.
db_name: neo4j
uri: bolt://localhost:7687
password: bpic2014
----------------------
If you have other credentials, please change them at: bpic14\config.yaml



### Clear Database

In [8]:
clear_database(db_connection)

2it [00:16,  9.11s/it, clear_db: took 13.14 seconds]

### Load the data as `(:Record)` nodes into Database
To import the records, we have defined how the records are structured per file in the BPIC14_DS.json file.
This file has been set in the conf_path.

In [9]:
load_data(db_connection, conf_path=conf_path)

13it [00:58,  4.42s/it, _filter_nodes for BPIC14Interaction: took 0.0 seconds]                      

In [10]:
get_graph_statistics(db_connection)


=== GRAPH STATISTICS ===

--- Node counts ---
(:Record)                      690622
(:Log)                         4

--- Relationship counts ---
[:CONTAINS]                    690622

--- Totals ---
Total nodes: 690626
Total relationships: 690622


In total, we should have loaded 690622 `(:Record)` nodes, 4 `(:Log)` nodes and 690622 `[:CONTAINS]` relationships.

We have now loaded all records from the four different logs as a `(:Record)` node.
Let's explore some of these `(:Record)` nodes.

In [11]:
def explore_records(_db_connection, _log_name, _limit):
    q_explore_records_str = '''
        MATCH (r:Record) <- [:CONTAINS] - (l:Log {name:$log_name})
        RETURN r limit $limit
    '''

    q_explore_records = Query(
        query_str = q_explore_records_str,
        parameters={
            'log_name': _log_name,
            'limit': _limit
        }
    )

    records = db_connection.exec_query(q_explore_records)
    for record in records:
        print(f"Record of {_log_name}: {record['r']['recordId']}")
        print(record['r'])

In [12]:
explore_records(db_connection, _log_name="BPIC14Incident.csv", _limit=2)

Record of BPIC14Incident.csv: 0_BPIC14Incident
{'serviceComponentAff': 'WBS000162', 'alertStatus': 'closed', 'ciNameCby': 'SUB000508', 'resolvedTime': neo4j.time.DateTime(2013, 11, 4, 13, 50, 27, 0, tzinfo=pytz.FixedOffset(60)), 'serviceComponentCBy': 'WBS000162', 'recordId': '0_BPIC14Incident', 'handleTimeHours': 3871.691111, 'urgency': 4, 'ciSubtypeCby': 'Web Based Application', 'numRelatedIncidents': 2, 'closeTime': neo4j.time.DateTime(2013, 11, 4, 13, 51, 17, 0, tzinfo=pytz.FixedOffset(60)), 'openTime': neo4j.time.DateTime(2012, 2, 5, 13, 32, 57, 0, tzinfo=pytz.FixedOffset(60)), 'impact': 4, 'index': 0, 'kmNumber': 'KM0000553', 'numRelatedInteractions': 1, 'ciTypeCby': 'subapplication', 'priority': 4, 'ciTypeAff': 'subapplication', 'relatedInteraction': 'SD0000007', 'closureCode': 'Other', 'numReassignments': 26, 'incidentId': 'IM0000004', 'category': 'incident', 'ciNameAff': 'SUB000508', 'ciSubtypeAff': 'Web Based Application', 'status': 'Closed'}
Record of BPIC14Incident.csv: 1_B

In [13]:
explore_records(db_connection, _log_name="BPIC14Interaction.csv", _limit=2)

Record of BPIC14Interaction.csv: 0_BPIC14Interaction
{'serviceComponentAff': 'WBS000125', 'interactionId': 'SD0000001', 'impact': 5, 'index': 0, 'kmNumber': 'KM0000987', 'priority': 4, 'ciTypeAff': 'application', 'recordId': '0_BPIC14Interaction', 'urgency': 4, 'relatedIncident': 'IM0000001', 'closeTime': neo4j.time.DateTime(2014, 2, 14, 9, 5, 0, 0, tzinfo=pytz.FixedOffset(60)), 'firstCallResolution': 'N', 'closureCode': 'Other', 'handleTimeSecs': 239, 'openTime': neo4j.time.DateTime(2011, 9, 9, 9, 23, 0, 0, tzinfo=pytz.FixedOffset(60)), 'category': 'incident', 'ciNameAff': 'SBA000243', 'ciSubtypeAff': 'Server Based Application', 'status': 'Closed'}
Record of BPIC14Interaction.csv: 1_BPIC14Interaction
{'serviceComponentAff': 'WBS000125', 'interactionId': 'SD0000002', 'impact': 4, 'index': 1, 'kmNumber': 'KM0000989', 'priority': 4, 'ciTypeAff': 'subapplication', 'recordId': '1_BPIC14Interaction', 'urgency': 4, 'relatedIncident': 'IM0000001', 'closeTime': neo4j.time.DateTime(2013, 12, 13

In [14]:
explore_records(db_connection, _log_name="Detail_Change.csv", _limit=2)

Record of Detail_Change.csv: 0_Detail_Change
{'serviceComponentAff': 'WBS000195', 'plannedEnd': neo4j.time.DateTime(2013, 12, 26, 0, 0, 0, 0, tzinfo=pytz.FixedOffset(60)), 'changeType': 'Release Type 11', 'actualEnd': neo4j.time.DateTime(2013, 12, 18, 16, 15, 0, 0, tzinfo=pytz.FixedOffset(60)), 'index': 0, 'cabApprovalNeeded': 'N', 'changeId': 'C00000003', 'plannedStart': neo4j.time.DateTime(2012, 8, 30, 7, 0, 0, 0, tzinfo=pytz.FixedOffset(60)), 'originatedFrom': 'Problem', 'ciTypeAff': 'hardware', 'recordId': '0_Detail_Change', 'emergencyChange': 'N', 'changeRecordOpenTime': neo4j.time.DateTime(2011, 9, 1, 9, 13, 0, 0, tzinfo=pytz.FixedOffset(60)), 'requestedEndDate': neo4j.time.DateTime(2012, 6, 1, 0, 0, 0, 0, tzinfo=pytz.FixedOffset(60)), 'riskAssessment': 'Minor Change', 'ciNameAff': 'HMD000002', 'ciSubtypeAff': 'MigratieDummy', 'actualStart': neo4j.time.DateTime(2013, 12, 18, 14, 0, 0, 0, tzinfo=pytz.FixedOffset(60)), 'changeRecordCloseTime': neo4j.time.DateTime(2013, 12, 18, 16, 

In [15]:
explore_records(db_connection, _log_name="Detail_Incident_Activity.csv", _limit=2)

Record of Detail_Incident_Activity.csv: 0_Detail_Incident_Activity
{'recordId': '0_Detail_Incident_Activity', 'interactionId': 'SD0000007', 'dateStamp': neo4j.time.DateTime(2013, 1, 7, 8, 17, 17, 0, tzinfo=pytz.FixedOffset(60)), 'assignmentGroup': 'TEAM0001', 'index': 0, 'kmNumber': 'KM0000553', 'incidentId': 'IM0000004', 'activityNumber': '001A3689763', 'incidentActivityType': 'Reassignment'}
Record of Detail_Incident_Activity.csv: 1_Detail_Incident_Activity
{'recordId': '1_Detail_Incident_Activity', 'interactionId': 'SD0000007', 'dateStamp': neo4j.time.DateTime(2013, 11, 4, 13, 41, 30, 0, tzinfo=pytz.FixedOffset(60)), 'assignmentGroup': 'TEAM0002', 'index': 1, 'kmNumber': 'KM0000553', 'incidentId': 'IM0000004', 'activityNumber': '001A5852941', 'incidentActivityType': 'Reassignment'}
